In [1]:
import numpy as np
import math as ma
import matplotlib.pyplot as plt
import astropy.units as u
import pickle, json, warnings, astropy, os
from matplotlib.colors import LogNorm
from astropy.io.votable import parse
%matplotlib inline

# Plotting Tool

In [2]:
def plotstuff(ax,x,y,xlabel,ylabel,marker,markersize,color,linestyle='none',linewidth=0,**kw):

    ax.plot(x,y,marker=marker,ms=markersize,color=color,ls=linestyle,lw=linewidth,**kw)
    ax.set_xlabel('%s'%xlabel,fontsize=20)
    ax.set_ylabel('%s'%ylabel,fontsize=20)
    return ax

## Load Completeness and Simulation Results

In [3]:
baseFolder = '/Users/rpatel/Dropbox/Research/WFIRST/EXOSIMSTesting/'
resultFolder = os.path.join(baseFolder,'SimResults')
scriptFolder = os.path.join(baseFolder,'scripts')
compFolder = os.path.join(baseFolder,'Completeness')
# ===============  LOAD JSON SCRIPT FILE =================

# jfile = 'template_WFIRST_KeplerLike.json'
# jfile = 'template_WFIRST_EarthTwinHabZone.json'
# jfile = 'template_WFIRST_KnownRV.json'

jfile = 'template_rpateltest_KnownRV.json'
scriptfile = os.path.join(scriptFolder,jfile)
script = open(scriptfile).read()
specs_from_file = json.loads(script)

# ===============  LOAD COMPLETENESS FILE ===============

# cfile = 'EarthTwinHabZone2.comp'
# cfile = 'KeplerLike1.comp'
cfile = 'KnownRVPlanetsd2a524f616563075bd74bff5e62577c9.comp'
fle = os.path.join(compFolder,cfile)
dataCOMP = pickle.load(open(fle,'rb'))

# ===============  LOAD SIMULATION RESULTS ===============

simresults = 'simresults_2yrs_5E+07stars_rpateltest_KnownRV.pickle'
basesim =  simresults.strip('simresults_').strip('.pickle')

simfile = os.path.join(resultFolder,simresults)
simr = pickle.load(open(simfile,'rb'))

print 'Upper level keys: \n\t', simr.keys()

Upper level keys: 
	['star_prop', 'DRM', 'synplanet_prop', 'etc_data', 'empplanet_prop', 'AllSpecs']


In [30]:
plt.figure()
plt.imshow(dataCOMP + 1,norm=LogNorm(),cmap='viridis')

## Stellar Data

In [23]:
sp = simr['star_prop']
print 'Star property keys: \n\t', sp.keys()

Star property keys: 
	['Bmag', 'dist', 'maxintTime', 'BV', 'Binary_Cut', 'Spec', 'rv', 'Vmag', 'Imag', 'Kmag', 'Lum', 'pmdec', 'MsEst', 'Name', 'nStars', 'Jmag', 'Hmag', 'parx', 'MsTrue', 'Umag', 'comp0', 'MV', 'coords', 'pmra']


## Simulated Planet Data

In [24]:
pd_s = simr['synplanet_prop']
if pd_s is not None:
    print 'Simulated Planet property keys: \n\t', pd_s.keys()
else: print 'No simulated planets from this simulation'

No simulated planets from this simulation


## Empirical Planet Data

In [25]:
pd_e = simr['empplanet_prop']
if pd_e is not None:
    print 'Empirical Planet property keys: \n\t', pd_e.keys()
else: print 'No empirical planets from this simulation'

Empirical Planet property keys: 
	['Rp', 'I', 'fEZ', 'v', 'sInds', 'O', 'p', 'r', 'Mp', 'w', 'nplans', 'e', 'plan2star', 'sma']


## DRM and Specs of simulation

In [10]:
DRM = simr['DRM']
AllSpecs = simr['AllSpecs']

In [31]:
# indices of targets observed in order including repeats
target_obsind = np.array([dt['target_ind'] for dt in DRM])

# indices of all planets detected for each star.
try: planet_detind = np.array([dt['plan_inds'] for dt in DRM])
except: print 'No planets detected?'

#arrival time array
arrival_time = np.array([dt['arrival_time'] for dt in DRM])

# angular distance of each detection ? mas
#det_wa = np.array([dt['det_WA'] for dt in DRM])

# status of any detections
det_status = np.array([dt['det_status'] for dt in DRM])

# detection of planets

target_observed = sp['Name'][target_obsind]
detstatus_array = np.array([ dt['det_status'] for dt in DRM])

## Target Plots

### Red circles are targets that were observed

In [46]:
# SKY DISTRIBUTION OF TARGETS
save_plots = False

dirsave = os.path.join(baseFolder,'SimPlots')

coords = sp['coords']
ra_rad, dec_rad = coords.ra.wrap_at(180*u.deg).radian, coords.dec.radian

BV, MV = sp['BV'], sp['MV']
dist = sp['dist']

plt.figure(figsize=(20,20))
plt.subplot(111,projection='aitoff')
plt.plot(ra_rad,dec_rad,'o',ms=5,alpha=0.3)
plt.plot(ra_rad[target_obsind],dec_rad[target_obsind],'ro',ms=5,mfc='none',mec='r',mew=3)
plt.title('Targets in Survey vs. Targets observed; %s' %basesim)    
if save_plots: plt.savefig(os.path.join(dirsave,'sky_targets_%s.png'%basesim))

# Observation Order
plt.figure(figsize=(20,20))
plt.subplot(111,projection='aitoff')
plt.plot(ra_rad,dec_rad,'bo',ms=4,alpha=0.3)
plt.plot(ra_rad[target_obsind],dec_rad[target_obsind],'r-',mew=2,lw=0.5,alpha=0.4)

for i,j in enumerate(target_obsind):
    if i == 0:
        plt.plot(ra_rad[j],dec_rad[j],'m*',ms=16)
    plt.text(ra_rad[j],dec_rad[j],str(i+1),color='k',fontsize=8)
plt.title('Order of observations; %s' %basesim)    
if save_plots: plt.savefig(os.path.join(dirsave,'sky_targets_order_%s.png'%basesim))


# CMD 
axbv = plt.figure(figsize=(6,6)).add_subplot(111)
axbv = plotstuff(axbv,BV,MV,'B-V','Mv',marker='o',color='c',markersize=6)
axbv.plot(BV[target_obsind],MV[target_obsind],'ro',markersize=6,mfc='none',mec='r',mew=2)
axbv.set_ylim(axbv.get_ylim()[::-1])
axbv.set_title('%s' %basesim)          
if save_plots: plt.savefig(os.path.join(dirsave,'target_CMD_%s.png'%basesim))

# DISTANCE VS. B-V
axbvd = plt.figure(figsize=(6,6)).add_subplot(111)
axbvd = plotstuff(axbvd,dist,BV,'Distance (pc)','B-V',marker='o',color='g',markersize=6)
axbvd.plot(dist[target_obsind],BV[target_obsind],'ro',ms=6,mfc='none',mec='r',mew=2)
axbvd.set_title('%s'%basesim)
if save_plots: plt.savefig(os.path.join(dirsave,'target_dist_v_B-V_%s.png'%basesim))


## Planet Plots

In [52]:
Rj = 69.911e6 #meters
Mj = 1.898e27 #Kg

Mp,Rp = pd_e['Mp'], pd_e['Rp']
sma,ecc = pd_e['sma'], pd_e['e']

plt.figure(figsize=(6,6))
plt.plot(Mp/Mj,Rp/Rj,'bo',ms=4)
plt.xlabel(r'Planet Mass ($M_J$)',fontsize=20)
plt.ylabel(r'Planet Radius ($R_J$)',fontsize=20)
plt.title('%s'%basesim)
plt.loglog()
if save_plots: plt.savefig(os.path.join(dirsave,'planet_R_v_M_%s.png'%basesim))

plt.figure(figsize=(6,6))
plt.hist(sma,bins=50,histtype='step',lw=3);
plt.xlabel('SMA (AU)')
plt.title('%s'%basesim)
if save_plots: plt.savefig(os.path.join(dirsave,'planet_SMA_distribution_%s.png'%basesim))

plt.figure(figsize=(6,6))
plt.hist(ecc,bins=50,histtype='step',lw=3)
plt.xlabel('eccentricity')
if save_plots: plt.savefig(os.path.join(dirsave,'planet_ecc_distribution_%s.png'%basesim))